# Using the `GridEditor`

[HoloGrid](https://github.com/pygridgen/holo-gridgen) is an interactive tool for the generation of orthonormal grids using [pygridgen](https://github.com/pygridgen/pygridgen) and the [HoloViz](holoviz.org) tool suite for use within [Jupyter notebooks](https://jupyter.org/) or deployable with [Panel](panel.pyviz.org). This notebook will demonstrate how you can use the primary class in `hologrid` called `GridEditor`.

You can install `hologrid` with conda into a Python 3.7 or 3.8 environment as follows:

```
conda install -c jlstevens -c conda-forge hologrid
```

With `hologrid` installed, first we import [GeoViews](http://geoviews.org/) and [GeoPandas](http://geopandas.org/) and load the GeoViews bokeh extension:



In [ ]:
import geopandas as gpd
import geoviews as gv
gv.extension('bokeh')

## `GridEditor` Basics 

Now we can import `GridEditor` from `hologrid`:

In [ ]:
from hologrid import GridEditor

This class can then be instantiated and given a handle.

In [ ]:
editor = GridEditor()

The main entrypoint on the instance is then the `.view()` method which we will call shortly. This method displays the Bokeh plot with the following set of tools in the side bar:

<img src='./assets/bokeh_tools.png' width='150px'></img>

By default, the 'Tap' tool is selected as indicated by the blue line on the side. To start defining a grid boundary, select the 'Point Draw' tool and click four times within the axes to define a square (the node marked with the triangle is the start node that was added first). Then hit the 'Generate mesh' button:

<img src='./assets/simple_example.png' width='900px'></img>

Have a go replicating the above screenshot in the area below:

In [ ]:
editor.view()

Note that you can do the following:

* You can move the nodes with the 'Point Draw' tool by click dragging them
* You can delete nodes with the 'Point Draw' tool by clicking them then hitting `Backspace`
* You can change the grid resolution by change the `Xres` and `Yres` values before hitting 'Generate mesh'
* You can hide any mesh you have generated by hitting the 'Hide mesh' button.

Now that you have defined a boundary, how can you access it programatically?

To get the corresponding geopandas `DataFrame`, simply use the `.boundary` property:


In [ ]:
editor.boundary

You'll notice that the values in the `x`, `y` and `geometry` columns are not latitudes and longitudes. This is because these values are in the [Web Mercator](https://en.wikipedia.org/wiki/Web_Mercator_projection) projection.


What if you want the last generated `pygridgen.grid.Gridgen` object? If available, this can be found using the `.grid` property:

In [ ]:
editor.grid

## Setting node polarities

You might have noticed in the previous example, that if you draw *five* boundary nodes, the 'Generate mesh' button is greyed out. This is because the total polarity of the nodes has to add up to *four* for the mesh generation to be available.

To define more complex boundaries, you therefore need to set the node polarity (`beta` values) appropriately. This is done by selecting on the 'Tap' tool and clicking on the nodes. Red nodes (by default) contribute +1 to polarity, blue nodes contribute -1 to polarity and the hollow nodes are neutral, adding zero to the total polarity.

The following screenshot show a more complex boundary you can define using both the 'Point Draw' and 'Tap' tools:

<img src='./assets/tap_tool.png' width='800px'></img>

Have a go defining this boundary in the view below (and inspecting `.boundary` afterwards, if you wish):

In [ ]:
complex_boundary = GridEditor()
complex_boundary.view()

Have a go playing with the 'Node size' and 'Edge width' sliders (which should be self-evident) as well as the 'Background' drop down that lets you select from a variety of map tile sources. Once you are happy with a boundary, you can also download the corresponding GeoJSON file by clicking the 'Download boundary.geojson' button.

## Inserting nodes into edges

You may have noticed one button that hasn't been mentioned yet, namely 'Insert points' which allows you to insert new points into an existing edge. To do this, you need to activate the 'Tap' tool again to use it for its second function of selecting edges. This the 'Tap' tool active, you can click an existing edge to select it (you can see when an edge is selected as the non-selected edges will then be muted in color). With an edge selected, you can now hit the 'Insert points' button to insert new nodes into the edge. As before, you can move these nodes around with the 'Point Draw' tool:

<img src='./assets/insert_points.png' width='800px'></img>

Note that you can reset your selection by clicking away from any node or boundary edge. Try inserting and moving around new nodes in one of the boundaries you have defined above.

## Setting options in the constructor

All the widgets and settings describe so far can be set using keywords in the `GridEditor` constructor. For instance, you can run:

In [ ]:
customized_editor = GridEditor(node_size=20, edge_width=4, xres=10, yres=40, background='EsriOceanBase')
customized_editor.view()

Which sets a larger (custom) node size and edge width as well as a custom resolution for the grids along the `x` and `y` directions. **Try defining a boundary over the world map in the cell above as it will help illustrate the serializing/restoring functionality later on!**

All these options are [`param`](https://param.holoviz.org/) `Parameters` and you can read more about the allowed values (and the docstrings for these settings) by running:


```python
gv.help(GridEditor)
```

Note that not all parameters are exposed as widgets such as `custom_background` and `focus`. We'll describe these features next:

## Setting a custom background

By setting the `custom_background` parameter, to a HoloViews or GeoViews element, you can use any background you wish. For instance, the following line (corresponding to a [GeoViews example](http://geoviews.org/index.html)) defines a set of polygons for the various countries of the world, colored by population (with Bokeh hover information enabled):

In [ ]:
custom_polygons = gv.Polygons(gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')), 
            vdims=['pop_est', ('name', 'Country')]).opts(
    tools=['hover'], width=600, alpha=0.4, cmap='plasma'
)

Now we simply supply this element in the constructor to the `custom_background` parameter:

In [ ]:
editor_with_custom_background = GridEditor(custom_background=custom_polygons, xres=25, yres=30, node_size=20)
editor_with_custom_background.view()

Note that the Bokeh hover tool is still enabled and the rest of the `GridEditor`'s functionality remains the same as before.

## Setting the `focus`

Once a boundary is defined, it is important to be able to tweak the orthonormal grid. One tool that makes this possible is `pygridgen`'s `Focus` class. Here is a simple example of defining a `Focus` instance:

In [ ]:
import pygridgen as pgg
focus = pgg.Focus()
focus.add_focus(0.50, 'y', factor=5, extent=0.25)

Now you can apply this focus the next the time 'Generate Mesh' button is hit by setting the `focus` parameter on the `GridEditor` instance. For example, to set a focus on the first example in the notebook we can execute:

In [ ]:
editor.focus = focus # Go back and hit 'Generate mesh' in the simple `editor` example above

<img src='./assets/focus.png' width='800px'></img>

To remove the `Focus` definition from the mesh generation process, simply set the parameter to `None`:

In [ ]:
editor.focus = None

## Saving and loading `GridEditor` state

You can easily serialize the state of `GridEditor` by using the `.data` property. This makes it easy to save a boundary editing session to disk (e.g as a JSON file for instance) and resume your work later. This serialized data can be passed to `GridEditor` as the first (optional) positional argument:

In [ ]:
serialized_data = customized_editor.data # Inspect this in the notebook (e.g use print)
restored_editor = GridEditor(serialized_data)
restored_editor.view()

Another way to load a boundary is to use the `from_geopandas` classmethod. If you drew a boundary in the `customized_editor` instance, you could also instantiate a new `GridEditor` with the same boundary using:

```python
restored_from_geopandas = GridEditor.from_geopandas(customized_editor.boundary)
```

## Serving a dashboard with `panel`

The output of the `view()` method is a panel object. This means it can be deployed as a dashboard by calling the `.servable` method (described in the [Panel User Guide](https://panel.holoviz.org/user_guide/Overview.html)). This means the simplest Panel dashboard you could create with `hologrid` is with:


```python
GridEditor().view().servable()
```